In [1]:
import aquamonitor as am

# The new functionality requires using the AM dev/test site
am.host = "https://test-aquamonitor.niva.no/"

ModuleNotFoundError: No module named 'aquamonitor'

# Query project-specific water chemistry data using the Aquamonitor API

AquaMonitor-Python is a Python library providing external access to Nivabasen, NIVA's internal water chemistry and ecology database.

## 1. Login

There are several login options:

 1. **Create a file named `.auth`** in the same folder as `AquaMonitor.py` with the format shown below. `Aquamonitor-Python` will then attempt to read your credentials from this file automatically whenever they are required. This is the easiest solution on secure systems, but **note that your username and password will be stored as plain text, so use with caution**.
 
            [Auth]
            username = YourUsername
            password = YourPassword
            
            
 2. **Use the `login()` function**, as shown below. This will create an access token that is valid for one day, which you can then pass to any subsequent queries (see below for examples)
 
 3. **Do nothing**. If you do not provide an access token and the `.auth` file cannot be found, you will be prompted to enter your username and password. Note that you will have to do this for *every* query, so in most cases one of the other options will be preferred    

In [2]:
# Get access token
token = am.login()

Please enter your credentials.


Username:  ···
Password:  ··············


## 2. List projects

The `get_projects` function returns a list of all projects in the database. You can search this dataframe to identify the correct project ID for the project you are interested in.

In [3]:
proj_df = am.get_projects(token=token)
print(f"{len(proj_df)} projects in the database.")
proj_df.head()

1300 projects in the database.


,ProjectId,ProjectCode,ProjectName,Description
0,1,NaN,xxx,NaN
1,55,NaN,PARCOM_Aquateam,NaN
2,56,NaN,PARCOM_NIVA,NaN
3,58,NaN,MAR_BOK,Fagdata
4,59,NaN,MAR_IMA,Fagdata


In [4]:
# Find the project code for the 2019 '1000 Lakes' survey
proj_df.query("ProjectName.str.contains('Nasjonal')", engine="python")

,ProjectId,ProjectCode,ProjectName,Description
1201,12433,190091,Nasjonal Innsjøundersøkelse 2019,1000-sjøer undersøkelse gjennomført i 2019


## 3. Get stations in project

Use `get_project_stations` to get a list of all stations associated with your project of interest.

**To do:** It would be great if this API endpoint could also return the geographic co-ordinates (lat/lon) for each station. At present, I think this can only be achieved by iterating over the `api/projects/{proj_id}/stations/{stn_id}/samplePoint` endpoint for each station, which is slow.

In [5]:
# Project of interest
proj_id = 12433

stn_df = am.get_project_stations(proj_id, token=token)
print(f"{len(stn_df)} stations in project.")
stn_df.head()

1003 stations in project.


,ProjectId,StationId,StationCode,StationName,Type
388,12433,26070,221-1-2,Langtjern,Innsjø
389,12433,26071,101-2-7,Hokksjøen,Innsjø
390,12433,26072,402-2-13,Sætertjern,Innsjø
391,12433,26073,419-1-25,Mjøgsjøen,Innsjø
392,12433,26074,425-2-2,Kottern,Innsjø


## 4. Get water chemistry

Use `get_project_chemistry` to extract all the water chemistry data for your project and time period of of interest.

In [6]:
%%time

# Period of interest
st_dt = "01.01.2019"
end_dt = "31.12.2019"

df = am.get_project_chemistry(proj_id, st_dt, end_dt, token=token, approved=True)
df.head()

CPU times: user 4.39 s, sys: 191 ms, total: 4.58 s
Wall time: 29.5 s


,ProjectId,ProjectName,StationId,StationCode,StationName,SampleDate,Depth2,ParameterName,Flag,Value,Unit,Approved,QuantificationLimit,DetectionLimit,Laboratory,MethodCode,MethodRef,Remark
10,12433,Nasjonal Innsjøundersøkelse 2019,26070,221-1-2,Langtjern,2019-10-25T00:00:00Z,NaN,Alkalitet,NaN,0.054,MMOLE_P_L,True,0.03,NaN,NIVA_LABWARE,NaN,C1-4 - NS-EN ISO 9963-1:1996 - NS-EN ISO/IEC 1...,NaN
13,12433,Nasjonal Innsjøundersøkelse 2019,26070,221-1-2,Langtjern,2019-10-25T00:00:00Z,NaN,Aluminium,NaN,280.000,UG_P_L,True,1.00,NaN,NIVA_LABWARE,NaN,EKSTERN_EF - ISO/IEC 17025:2005 SWEDAC 1125,NaN
477,12433,Nasjonal Innsjøundersøkelse 2019,26070,221-1-2,Langtjern,2019-10-25T00:00:00Z,NaN,"Aluminium, ikke labil",NaN,130.000,UG_P_L,True,5.00,1.5,NIVA_LABWARE,NaN,E3-2 - NS-EN ISO/IEC 17025:2005 NA TEST 009,NaN
243,12433,Nasjonal Innsjøundersøkelse 2019,26070,221-1-2,Langtjern,2019-10-25T00:00:00Z,NaN,"Aluminium, reaktivt",NaN,150.000,UG_P_L,True,5.00,1.5,NIVA_LABWARE,NaN,E3-2 - NS-EN ISO/IEC 17025:2005 NA TEST 009,NaN
183,12433,Nasjonal Innsjøundersøkelse 2019,26070,221-1-2,Langtjern,2019-10-25T00:00:00Z,NaN,Ammonium,NaN,6.000,UG_N_L,True,2.00,NaN,NIVA_LABWARE,NaN,C4-4 - NS-EN ISO 10304-1:2009 (Anioner) NS-EN ...,NaN
